# Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter)
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

### If on runpod / vast.ai / etc, spin up an A6000 or A100 pod using a Stable Diffusion template with Jupyter pre-installed.

## Check GPU

In [ ]:
# mostly for Google Colab -- you should know what you're running on Runpod / Vast.ai / etc
from IPython.display import HTML
from subprocess import getoutput
s = getoutput('nvidia-smi')
print(s)
# or simply
#!nvidia-smi -L

## Installation

In [ ]:
# GIT THE REPO
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion

# OR THE ORIGINAL
# !git clone https://github.com/XavierXiao/Dreambooth-Stable-Diffusion

## Build Environment

In [ ]:
#BUILD ENV
%cd /workspace/Dreambooth-Stable-Diffusion
!pip install omegaconf einops pytorch-lightning==1.6.5 test-tube transformers kornia -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .

## Generate regularization images (you should change the code below)

This repo simultaneously trains both your token **and** retrains your class.

From cursory testing, it does not seem like the number of reg images affects the model too much.

However, it does affect your class greatly.

In [ ]:
#GENERATE 50 images
!python scripts/stable_txt2img.py --seed 10 --ddim_eta 0.0 --n_samples 1 --n_iter 50 --scale 10.0 --ddim_steps 50  --ckpt /workspace/stable-diffusion-webui/model.ckpt \
--prompt "person"

# IMPORTANT!! Change "person" above, and on the cell below:

## Upload your samples

Drag your training images into /workspace/Dreambooth-Stable-Diffusion/training_samples

At the moment, it seems like 10-20 images of someone's face is enough.

## Prep Training Variables

Navigate to:
/workspace/Dreambooth-Stable-Diffusion/ldm/data/personalized.py

Change "sks" in line 11 to whatever you want your token to be.

e.g. I changed mine to
```python
training_templates_smallest = [
    'joepenna {}',
]
```

The last line of this file trains for `3000` steps.
/workspace/Dreambooth-Stable-Diffusion/configs/stable-diffusion/v1-finetune_unfrozen.yaml

If training a person or subject, keep an eye on your project's `/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `/images/train/samples_gs-00xxxx` generations.

## Start Training (you should also change the code below)

In [ ]:
# START THE TRAINING
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "/workspace/stable-diffusion-webui/model.ckpt" \
 --reg_data_root "/workspace/Dreambooth-Stable-Diffusion/outputs/txt2img-samples/samples" \
 -n "project_name" \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Stable-Diffusion/training_imgs" \
 --class_word "person" # << match this word to the class word from regularization images above

## Generate Images With Trained Model

Be sure to change `PROJECT_PATH` below to match the folder in:
/workspace/Dreambooth-Stable-Diffusion/logs

Also change `sks person` in the prompt to the token *and* class_word.

In [ ]:
!python scripts/stable_txt2img.py \
--ddim_eta 0.0 --n_samples 1 --n_iter 4 \
--scale 7.0 --ddim_steps 60 --ckpt "/workspace/Dreambooth-Stable-Diffusion/logs/PROJECT_PATH_GOES_HERE/last.ckpt" \
--prompt "sks person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"

Trained generated images at /workspace/Dreambooth-Stable-Diffusion/outputs/